In [1]:
import sys
import os
from pathlib import Path
from glob import glob

In [2]:
sys.path.append('..')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

In [4]:
from experiments.eval_features import eval_features

In [5]:
SEED = 188283
PATH_ROOT = Path(os.getcwd()).absolute().parent
PATH_RESULTS = os.path.join(PATH_ROOT, 'results', f'exp{SEED}')
PATH_OUTPUT = os.path.join(PATH_RESULTS, 'roc')

print('INPUTS:', PATH_RESULTS)

INPUTS: /home/lukec/workspace/baard_v4/results/exp188283


In [6]:
PATH_OUTPUT = os.path.join(PATH_RESULTS, 'roc')
print('OUTPUTS:', PATH_OUTPUT)

if not os.path.exists(PATH_OUTPUT):
    os.makedirs(PATH_OUTPUT)

OUTPUTS: /home/lukec/workspace/baard_v4/results/exp188283/roc


In [7]:
def get_result_one_detector(detector_name, data_name, attack_name, eps, path_output=PATH_OUTPUT):
    path_input = os.path.join(PATH_RESULTS, data_name, detector_name, attack_name)
    # NOTE: LID and ML-LOO need to call proba.
    if detector_name == 'MLLooDetector' or detector_name == 'LIDDetector':
        _detector_name = detector_name + '(proba)'
    else:
        _detector_name = detector_name
    file_adv = f'{_detector_name}-{data_name}-{attack_name}-{eps}.pt'
    file_clean = f'{_detector_name}-{data_name}-{attack_name}-clean.pt'
    _, df_auc_tpr = eval_features(path_input, path_output, file_clean, file_adv)
    df_auc_tpr = (df_auc_tpr * 100).round(1)
    df_auc_tpr['data'] = data_name
    df_auc_tpr['detector'] = detector_name
    df_auc_tpr['attack'] = attack_name
    df_auc_tpr['eps'] = eps
    df_auc_tpr = df_auc_tpr[['data', 'detector', 'attack', 'eps', 'auc', '1fpr', '5fpr', '10fpr']]
    return df_auc_tpr


In [8]:
detectors = [
    'RegionBasedClassifier',
    'FeatureSqueezingDetector',
    'LIDDetector',
    'OddsAreOddDetector',
    'MLLooDetector',
    'PNDetector',
    'BAARD',
]
attacks = ['APGD-L2', 'APGD-Linf', 'CW2-L2']
detector_mapping = {
    'RegionBasedClassifier': 'RC',
    'FeatureSqueezingDetector': 'FS',
    'LIDDetector': 'LID',
    'OddsAreOddDetector': 'Odds',
    'MLLooDetector': 'ML-LOO',
    'PNDetector': 'PN',
    'BAARD': 'BAARD',
}

In [9]:
data_name = 'MNIST'
eps_list = {
    'APGD-L2': [4.0, 8.0],
    'APGD-Linf': [0.22, 0.66],
    'CW2-L2': [0]
}
mnist_eps_mapping = {
    4.0: 'Low',
    8.0: 'High',
    0.22: 'Low',
    0.66: 'High',
}

df_greybox_mnist = pd.DataFrame()
for attack_name in attacks:
    for eps in eps_list[attack_name]:
        for detector_name in detectors:
            _df_row = get_result_one_detector(detector_name, data_name, attack_name, eps)
            df_greybox_mnist = pd.concat([df_greybox_mnist, _df_row])

df_greybox_mnist['detector'] = df_greybox_mnist['detector'].map(detector_mapping)
df_greybox_mnist['eps'] = df_greybox_mnist['eps'].map(mnist_eps_mapping)

df_greybox_mnist.to_csv(os.path.join(PATH_OUTPUT, f'greybox_{data_name}.csv'))

print(df_greybox_mnist[['data', 'detector', 'attack', 'eps', 'auc', '5fpr']])


    data detector     attack   eps    auc   5fpr
0  MNIST       RC    APGD-L2   Low   50.8    0.0
0  MNIST       FS    APGD-L2   Low   75.1   30.6
0  MNIST      LID    APGD-L2   Low   43.4    7.1
0  MNIST     Odds    APGD-L2   Low   96.2   78.4
0  MNIST   ML-LOO    APGD-L2   Low   94.1   72.5
0  MNIST       PN    APGD-L2   Low   56.2   10.9
0  MNIST    BAARD    APGD-L2   Low   93.0   56.9
0  MNIST       RC    APGD-L2  High   51.2    0.0
0  MNIST       FS    APGD-L2  High   66.9    3.6
0  MNIST      LID    APGD-L2  High   82.0   45.9
0  MNIST     Odds    APGD-L2  High   96.3   80.5
0  MNIST   ML-LOO    APGD-L2  High  100.0  100.0
0  MNIST       PN    APGD-L2  High   57.2    6.6
0  MNIST    BAARD    APGD-L2  High   97.4   85.1
0  MNIST       RC  APGD-Linf   Low   73.3   49.6
0  MNIST       FS  APGD-Linf   Low   99.8   99.6
0  MNIST      LID  APGD-Linf   Low   61.3   19.3
0  MNIST     Odds  APGD-Linf   Low   98.7   97.3
0  MNIST   ML-LOO  APGD-Linf   Low   99.7   99.0
0  MNIST       PN  A

In [10]:
data_name = 'CIFAR10'
eps_list = {
    'APGD-L2': [0.3, 3.0],
    'APGD-Linf': [0.01, 0.1],
    'CW2-L2': [0]
}
cifar10_eps_mapping = {
    0.3: 'Low',
    3.0: 'High',
    0.01: 'Low',
    0.1: 'High',
}

df_greybox_cifar10 = pd.DataFrame()
for attack_name in attacks:
    for eps in eps_list[attack_name]:
        for detector_name in detectors:
            _df_row = get_result_one_detector(detector_name, data_name, attack_name, eps)
            df_greybox_cifar10 = pd.concat([df_greybox_cifar10, _df_row])

df_greybox_cifar10['detector'] = df_greybox_cifar10['detector'].map(detector_mapping)
df_greybox_cifar10['eps'] = df_greybox_cifar10['eps'].map(cifar10_eps_mapping)

df_greybox_cifar10.to_csv(os.path.join(PATH_OUTPUT, f'greybox_{data_name}.csv'))

print(df_greybox_cifar10[['data', 'detector', 'attack', 'eps', 'auc', '5fpr']])


      data detector     attack   eps    auc   5fpr
0  CIFAR10       RC    APGD-L2   Low   56.8   13.4
0  CIFAR10       FS    APGD-L2   Low   95.1   78.7
0  CIFAR10      LID    APGD-L2   Low   73.2   31.5
0  CIFAR10     Odds    APGD-L2   Low   97.5   95.6
0  CIFAR10   ML-LOO    APGD-L2   Low   56.5   15.7
0  CIFAR10       PN    APGD-L2   Low   76.0   19.5
0  CIFAR10    BAARD    APGD-L2   Low   72.6   23.6
0  CIFAR10       RC    APGD-L2  High   55.0    0.0
0  CIFAR10       FS    APGD-L2  High   82.0   43.0
0  CIFAR10      LID    APGD-L2  High   99.0   95.4
0  CIFAR10     Odds    APGD-L2  High   80.3    2.9
0  CIFAR10   ML-LOO    APGD-L2  High   98.8   96.4
0  CIFAR10       PN    APGD-L2  High   57.7   11.5
0  CIFAR10    BAARD    APGD-L2  High  100.0  100.0
0  CIFAR10       RC  APGD-Linf   Low   50.2    3.7
0  CIFAR10       FS  APGD-Linf   Low   95.4   75.3
0  CIFAR10      LID  APGD-Linf   Low   83.4   45.6
0  CIFAR10     Odds  APGD-Linf   Low   98.1   96.0
0  CIFAR10   ML-LOO  APGD-Linf 